In [3]:
import math
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import glob
from sklearn.utils.class_weight import compute_class_weight
from joblib import dump
import os


# Assuming P24, P48, and Adult datasets are loaded as numpy arrays
# Define the directory path where your data is located
directory_path = '/n/sci/SCI-004375-NYUDATA/Filippo/Multiome'
os.chdir(directory_path)
print("Current working directory:", os.getcwd())

# Loading in shuffled datasets with full file paths
Adult = np.load(os.path.join(directory_path, "AdultData_Shuffled.npy"))
ClustersAdult = np.load(os.path.join(directory_path, "AdultIDs_Shuffled.npy"))

P24 = np.load(os.path.join(directory_path, "P24Data_Shuffled.npy"))
ClustersP24 = np.load(os.path.join(directory_path, "P24IDs_Shuffled.npy"))

P48 = np.load(os.path.join(directory_path, "P48Data_Shuffled.npy"))
ClustersP48 = np.load(os.path.join(directory_path, "P48IDs_Shuffled.npy"))
# Calculate 90% split point for each dataset
split_P24 = int(len(P24) * 0.9)
split_P48 = int(len(P48) * 0.9)
split_Adult = int(len(Adult) * 0.9)

# Print the split points
print("90% split point for P24:", split_P24)
print("90% split point for P48:", split_P48)
print("90% split point for Adult:", split_Adult)



2024-07-09 13:54:17.693399: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-09 13:54:17.739946: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-09 13:54:18.633390: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Current working directory: /n/sci/SCI-004375-NYUDATA/Filippo/Multiome
90% split point for P24: 58390
90% split point for P48: 54368
90% split point for Adult: 42648


In [1]:
import math
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import glob
from sklearn.utils.class_weight import compute_class_weight
from joblib import dump
import os

# Define the directory path where your data is located
directory_path = '/n/sci/SCI-004375-NYUDATA/Filippo/Multiome'
os.chdir(directory_path)
print("Current working directory:", os.getcwd())

# Loading in shuffled datasets with full file paths
Adult = np.load(os.path.join(directory_path, "AdultData_Shuffled.npy"))
ClustersAdult = np.load(os.path.join(directory_path, "AdultIDs_Shuffled.npy"))

P24 = np.load(os.path.join(directory_path, "P24Data_Shuffled.npy"))
ClustersP24 = np.load(os.path.join(directory_path, "P24IDs_Shuffled.npy"))

P48 = np.load(os.path.join(directory_path, "P48Data_Shuffled.npy"))
ClustersP48 = np.load(os.path.join(directory_path, "P48IDs_Shuffled.npy"))

# Set the input shapes based on the dataset shapes
inputAdult_shape = 742
inputP24_shape = 632
inputP48_shape = 752

# Encoder
Clusters = np.concatenate((ClustersAdult, ClustersP24, ClustersP48))
encoder = LabelEncoder()
encoder.fit(Clusters)
Clusters = encoder.transform(Clusters)
Clusters = to_categorical(Clusters)
dump(encoder, 'Encoder_multinet.joblib')  # Saving encoder file joblib 0.17.0

ClustersAdult = Clusters[:len(ClustersAdult), ]
ClustersP24 = Clusters[len(ClustersAdult):len(ClustersAdult) + len(ClustersP24), ]
ClustersP48 = Clusters[len(ClustersAdult) + len(ClustersP24):len(ClustersAdult) + len(ClustersP24) + len(ClustersP48), ]

# Normalization
meanAdult = Adult.mean(axis=0)
Adult -= meanAdult
np.save('meanAdult.npy', meanAdult)  # Save the mean for normalization

meanP24 = P24.mean(axis=0)
P24 -= meanP24
np.save('meanP24.npy', meanP24)  # Save the mean for normalization

meanP48 = P48.mean(axis=0)
P48 -= meanP48
np.save('meanP48.npy', meanP48)  # Save the mean for normalization

y_integers = np.argmax(Clusters, axis=1)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_integers), y=y_integers)
d_class_weights = dict(enumerate(class_weights))

K.clear_session()

# Define input layers with the correct shapes
inputAdult = Input(shape=(inputAdult_shape,))  # Shape input # of cluster marker genes
inputP24 = Input(shape=(inputP24_shape,))
inputP48 = Input(shape=(inputP48_shape,))

def create_model(input_tensor):
    x = layers.Dropout(0.2)(input_tensor)
    x = layers.Dense(200, kernel_regularizer=regularizers.l2(0.002), activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output_tensor = layers.Dense(259, kernel_regularizer=regularizers.l2(0.002), activation='softmax')(x)  # Updated output shape
    return Model(input_tensor, output_tensor)

modelAdult = create_model(inputAdult)
modelP24 = create_model(inputP24)
modelP48 = create_model(inputP48)

# Define a new optimizer for each model
def get_optimizer():
    return tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.9, nesterov=True)

modelAdult.compile(optimizer=get_optimizer(), loss='categorical_crossentropy', metrics=['accuracy'])
modelP24.compile(optimizer=get_optimizer(), loss='categorical_crossentropy', metrics=['accuracy'])
modelP48.compile(optimizer=get_optimizer(), loss='categorical_crossentropy', metrics=['accuracy'])

def step_decay(epoch):
    initial_lrate = 0.05
    drop = 0.5
    epochs_drop = 3
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

def find_last(files):
    epochs = []
    for k in files:
        splits = k.split("_")
        finalepoch = splits[2].split(".")[0]
        epochs.append(int(finalepoch))
    maxepoch = max(epochs)
    return str(maxepoch).zfill(2)

RunTable = pd.DataFrame(columns=['Run', 'Epoch', 'Stage', 'Hist_val_loss', 'Hist_val_acc', 'Hist_loss', 'Hist_acc'])
EvalTable = pd.DataFrame(columns=['Run', 'EpochSave', 'Stage', 'Filename', 'Manual_Val_Loss', 'Manual_Val_Acc', 'Manual_Train_Loss', 'Manual_Train_Acc'])

best_manual_val_loss = float('inf')
best_manual_val_acc = 0

# Define the split points for new datasets
split_P24 = 58390
split_P48 = 54368
split_Adult = 42648

eps = 30
for j in range(1, 15):  # Changed to 14 runs
    # Print the current run number
    print(f"Starting run {j}")

    # Initialize lists to store metrics for each run
    val_loss_Adult = []
    val_acc_Adult = []
    loss_Adult = []
    acc_Adult = []
    val_loss_P24 = []
    val_acc_P24 = []
    loss_P24 = []
    acc_P24 = []
    val_loss_P48 = []
    val_acc_P48 = []
    loss_P48 = []
    acc_P48 = []

    callbacks_list1 = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint(filepath=f'autosaveAdult_{j}_{{epoch:02d}}.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
        tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=1),
    ]
    callbacks_list2 = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint(filepath=f'autosaveP24_{j}_{{epoch:02d}}.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
        tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=1),
    ]
    callbacks_list3 = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint(filepath=f'autosaveP48_{j}_{{epoch:02d}}.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
        tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=1),
    ]

    for i in range(eps):
        # Print the current epoch number
        print(f"Run {j}, Epoch {i+1}/{eps}")

        history = modelAdult.fit(Adult, ClustersAdult, epochs=i+1, initial_epoch=i, batch_size=128, callbacks=callbacks_list1, validation_data=(Adult[split_Adult:], ClustersAdult[split_Adult:]), shuffle=True, class_weight=d_class_weights)

        val_loss_Adult.append(history.history['val_loss'])
        val_acc_Adult.append(history.history['val_accuracy'])
        loss_Adult.append(history.history['loss'])
        acc_Adult.append(history.history['accuracy'])

        RunTable = pd.concat([RunTable, pd.DataFrame({
            'Run': [j], 'Epoch': [i+1], 'Stage': ['Adult'],
            'Hist_val_loss': [history.history['val_loss']], 'Hist_val_acc': [history.history['val_accuracy']],
            'Hist_loss': [history.history['loss']], 'Hist_acc': [history.history['accuracy']]
        })], ignore_index=True)

        # Copying weights from Adult to P24
        w = modelAdult.layers[3].get_weights()
        modelP24.layers[3].set_weights(w)
        modelP24.compile(optimizer=get_optimizer(), loss='categorical_crossentropy', metrics=['accuracy'])  # Recompile the model
        history = modelP24.fit(P24, ClustersP24, epochs=i+1, initial_epoch=i, batch_size=128, callbacks=callbacks_list2, validation_data=(P24[split_P24:], ClustersP24[split_P24:]), shuffle=True, class_weight=d_class_weights)

        val_loss_P24.append(history.history['val_loss'])
        val_acc_P24.append(history.history['val_accuracy'])
        loss_P24.append(history.history['loss'])
        acc_P24.append(history.history['accuracy'])

        RunTable = pd.concat([RunTable, pd.DataFrame({
            'Run': [j], 'Epoch': [i+1], 'Stage': ['P24'],
            'Hist_val_loss': [history.history['val_loss']], 'Hist_val_acc': [history.history['val_accuracy']],
            'Hist_loss': [history.history['loss']], 'Hist_acc': [history.history['accuracy']]
        })], ignore_index=True)

        # Copying weights from P24 to P48
        w = modelP24.layers[3].get_weights()
        modelP48.layers[3].set_weights(w)
        modelP48.compile(optimizer=get_optimizer(), loss='categorical_crossentropy', metrics=['accuracy'])  # Recompile the model
        history = modelP48.fit(P48, ClustersP48, epochs=i+1, initial_epoch=i, batch_size=128, callbacks=callbacks_list3, validation_data=(P48[split_P48:], ClustersP48[split_P48:]), shuffle=True, class_weight=d_class_weights)

        val_loss_P48.append(history.history['val_loss'])
        val_acc_P48.append(history.history['val_accuracy'])
        loss_P48.append(history.history['loss'])
        acc_P48.append(history.history['accuracy'])

        RunTable = pd.concat([RunTable, pd.DataFrame({
            'Run': [j], 'Epoch': [i+1], 'Stage': ['P48'],
            'Hist_val_loss': [history.history['val_loss']], 'Hist_val_acc': [history.history['val_accuracy']],
            'Hist_loss': [history.history['loss']], 'Hist_acc': [history.history['accuracy']]
        })], ignore_index=True)

        # Copying weights from P48 back to Adult
        w = modelP48.layers[3].get_weights()
        modelAdult.layers[3].set_weights(w)
        modelAdult.compile(optimizer=get_optimizer(), loss='categorical_crossentropy', metrics=['accuracy'])  # Recompile the model
        RunTable.to_csv('RunTable.csv')

    # Print a summary after completing all epochs for the current run
    print(f"Completed run {j} with {eps} epochs")

    # Evaluate and save the best models
    for stage, files, model, data, clusters, data_split in [
        ('Adult', glob.glob(f'autosaveAdult_{j}_*.keras'), modelAdult, Adult, ClustersAdult, split_Adult),
        ('P24', glob.glob(f'autosaveP24_{j}_*.keras'), modelP24, P24, ClustersP24, split_P24),
        ('P48', glob.glob(f'autosaveP48_{j}_*.keras'), modelP48, P48, ClustersP48, split_P48)
    ]:
        topepoch = find_last(files)
        topfile = f'autosave{stage}_{j}_{topepoch}.keras'
        print(topfile)
        try:
            top_model = load_model(topfile)
            resultsTrain = top_model.evaluate(data[:data_split, ], clusters[:data_split, ])
            resultsVal = top_model.evaluate(data[data_split:, ], clusters[data_split:, ])
            EvalTable = pd.concat([EvalTable, pd.DataFrame({
                'Run': [j], 'EpochSave': [topepoch], 'Stage': [stage],
                'Filename': [topfile], 'Manual_Val_Loss': [resultsVal[0]], 'Manual_Val_Acc': [resultsVal[1]],
                'Manual_Train_Loss': [resultsTrain[0]], 'Manual_Train_Acc': [resultsTrain[1]]
            })], ignore_index=True)
            EvalTable.to_csv('EvalTable.csv')

            # Check if this is the best model so far based on validation loss and accuracy
            if resultsVal[0] < best_manual_val_loss or (resultsVal[0] == best_manual_val_loss and resultsVal[1] > best_manual_val_acc):
                best_manual_val_loss = resultsVal[0]
                best_manual_val_acc = resultsVal[1]
                best_model_path = f'best_model_{stage}_run_{j}.keras'
                top_model.save(best_model_path)
                print(f"New best model saved: {best_model_path}")

        except ValueError as e:
            print(f"Error loading model {topfile}: {e}")
            break  # Stop further execution on error

    # Re-save the model to ensure compatibility with the current TensorFlow version
    modelAdult.save(f'modelAdult_run_{j}.keras')
    modelP24.save(f'modelP24_run_{j}.keras')
    modelP48.save(f'modelP48_run_{j}.keras')

# Visualize the tables after all runs
RunTable = pd.read_csv('RunTable.csv')
print("RunTable:")
print(RunTable)

# Load and print the consolidated evaluation table
EvalTable = pd.read_csv('EvalTable.csv')
print("EvalTable:")
print(EvalTable)


2024-07-11 13:16:09.609362: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 13:16:09.653132: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 13:16:12.201466: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Current working directory: /n/sci/SCI-004375-NYUDATA/Filippo/Multiome


2024-07-11 13:16:17.445269: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38141 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:e3:00.0, compute capability: 8.0


Starting run 1
Run 1, Epoch 1/30

Epoch 1: LearningRateScheduler setting learning rate to 0.05.


I0000 00:00:1720721778.440549 2586610 service.cc:145] XLA service 0x7fd37c007d60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720721778.440581 2586610 service.cc:153]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-11 13:16:18.461797: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-11 13:16:19.348876: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


...


Epoch 1: val_accuracy improved from -inf to 0.81874, saving model to autosaveAdult_1_01.keras
371/371 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.3843 - loss: 4.7152 - val_accuracy: 0.8187 - val_loss: 2.9388 - learning_rate: 0.0500

Epoch 1: LearningRateScheduler setting learning rate to 0.05.
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4570 - loss: 4.0862  

I0000 00:00:1720721795.207732 2586805 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_65', 300 bytes spill stores, 300 bytes spill loads

I0000 00:00:1720721795.280023 2586800 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_65', 24 bytes spill stores, 24 bytes spill loads

I0000 00:00:1720721795.700615 2586801 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_65', 40 bytes spill stores, 40 bytes spill loads




Epoch 1: val_accuracy improved from -inf to 0.84772, saving model to autosaveP24_1_01.keras
507/507 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.4573 - loss: 4.0848 - val_accuracy: 0.8477 - val_loss: 2.2092 - learning_rate: 0.0500

Epoch 1: LearningRateScheduler setting learning rate to 0.05.
472/472 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4686 - loss: 6.4613  
Epoch 1: val_accuracy improved from -inf to 0.88297, saving model to autosaveP48_1_01.keras
472/472 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.4688 - loss: 6.4666 - val_accuracy: 0.8830 - val_loss: 9.4000 - learning_rate: 0.0500
Run 1, Epoch 2/30

Epoch 2: LearningRateScheduler setting learning rate to 0.05.
Epoch 2/2
371/371 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6407 - loss: 3.6684  
Epoch 2: val_accuracy improved from 0.81874 to 0.82781, saving model to autosaveAdult_1_02.keras
371/371 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6407 - loss: 3.6685 - val_accuracy: 0.8278 - val_loss: 2.8421 - learnin

/tmp/ipykernel_2584682/1666383140.py:229: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  EvalTable = pd.concat([EvalTable, pd.DataFrame({


1825/1825 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9084 - loss: 1.1017
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9096 - loss: 1.1142
New best model saved: best_model_P24_run_1.keras
autosaveP48_1_15.keras
1699/1699 ━━━━━━━━━━━━━━━━━━━━ 2s 620us/step - accuracy: 0.9798 - loss: 0.8564
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9829 - loss: 0.8532
New best model saved: best_model_P48_run_1.keras
Starting run 2
Run 2, Epoch 1/30

Epoch 1: LearningRateScheduler setting learning rate to 0.05.
371/371 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7506 - loss: 1.8925  
Epoch 1: val_accuracy improved from -inf to 0.81177, saving model to autosaveAdult_2_01.keras
371/371 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7504 - loss: 1.8939 - val_accuracy: 0.8118 - val_loss: 2.5325 - learning_rate: 0.0500

Epoch 1: LearningRateScheduler setting learning rate to 0.05.
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7701 - loss: 1.8575  
Epoch 1: val_accuracy imp